# Strompreise

In [1]:
import time
from typing import Optional, Tuple, List
import datetime as dt
import pandas as pd
import pytz
import requests
import streamlit as st
import plotly.graph_objects as go

In [2]:
resolution_choice = "quarterly"

In [14]:
# ---------------------------------------------------------
# SMARD Loader
# ---------------------------------------------------------
SERIES_ID = "4169"
REGION_CANDIDATES = ["DE-LU", "DE"]
SMARD_BASE = "https://www.smard.de/app"
HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; Streamlit-SMARD/1.0)"}
_last_tried: List[str] = []

class SmardError(Exception):
    pass

def _safe_get_json(url: str, timeout: int = 30) -> dict:
    _last_tried.append(url)
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    if r.status_code != 200:
        raise SmardError(f"HTTP {r.status_code} für {url}")
    try:
        return r.json()
    except Exception:
        snippet = (r.text or "")[:160].replace("\n", " ")
        ctype = r.headers.get("Content-Type", "")
        raise SmardError(f"Kein JSON von {url}. Content-Type='{ctype}', Antwort: '{snippet}...'")

def _get_index(region: str, resolution: str) -> list[int]:
    url = f"{SMARD_BASE}/chart_data/{SERIES_ID}/{region}/index_{resolution}.json"
    data = _safe_get_json(url)
    ts = data.get("timestamps") or []
    if not ts:
        raise SmardError(f"Keine timestamps in index_{resolution}.json ({region})")
    return ts

def _try_load_series(region: str, resolution: str, ts: int) -> Optional[pd.DataFrame]:
    for path in ["table_data", "chart_data"]:
        url = f"{SMARD_BASE}/{path}/{SERIES_ID}/{region}/{SERIES_ID}_{region}_{resolution}_{ts}.json"
        try:
            data = _safe_get_json(url)
            series = data.get("series")
            if series:
                return pd.DataFrame(series, columns=["ts_ms", "eur_per_mwh"])
        except SmardError:
            continue
    return None

# @st.cache_data(ttl=900)
def load_smard_series(prefer_resolution: str = "quarterhour", max_backsteps: int = 12) -> Tuple[pd.DataFrame, str, str]:
    resolutions = [prefer_resolution] + ([r for r in ["hour"] if r != prefer_resolution])
    for region in REGION_CANDIDATES:
        for resolution in resolutions:
            try:
                idx = _get_index(region, resolution)
            except SmardError:
                continue
            for ts in reversed(idx[-(max_backsteps + 1):]):
                df = _try_load_series(region, resolution, ts)
                if df is not None and not df.empty:
                    return df, resolution, region
                time.sleep(0.15)
    raise SmardError("Keine gültige SMARD-Datei gefunden (region/auflösung/ts).")

In [29]:
df_raw, used_resolution, used_region = load_smard_series(prefer_resolution=resolution_choice, max_backsteps=12)

In [30]:
df_raw

,ts_ms,eur_per_mwh
0,1760306400000,93.45
1,1760310000000,90.26
2,1760313600000,90.83
3,1760317200000,89.12
4,1760320800000,93.45
...,...,...
163,1760893200000,NaN
164,1760896800000,NaN
165,1760900400000,NaN
166,1760904000000,NaN


In [26]:
# ---------------------------------------------------------
# Data preparation
# ---------------------------------------------------------
tz_berlin = pytz.timezone("Europe/Berlin")
df_raw["ts"] = pd.to_datetime(df_raw["ts_ms"], unit="ms", utc=True).dt.tz_convert("Europe/Berlin")
df_raw["ct_per_kwh"] = df_raw["eur_per_mwh"] * 0.1

# ---------------------------------------------------------
# Zeitfenster: von aktuellem Mittag (12:00) bis nächstes Mittag (12:00)
# ---------------------------------------------------------
tz_berlin = pytz.timezone("Europe/Berlin")
now = dt.datetime.now(tz=tz_berlin)
today = now.date()

# define nominal window 12:00→12:00
start_window = tz_berlin.localize(dt.datetime.combine(today, dt.time(12, 0)))
end_window = start_window + dt.timedelta(days=1)

df_raw["ts"] = pd.to_datetime(df_raw["ts_ms"], unit="ms", utc=True).dt.tz_convert("Europe/Berlin")
df_raw["ct_per_kwh"] = df_raw["eur_per_mwh"] * 0.1

# Filter: keep data that falls inside that nominal 24-h window
df = df_raw[(df_raw["ts"] >= start_window - dt.timedelta(hours=12)) &
            (df_raw["ts"] < end_window + dt.timedelta(hours=12))].copy()

# ensure we actually cover the full delivery range; extend end to +12 h after last timestamp if needed
if not df.empty:
    last_ts = df["ts"].max()
    if last_ts < end_window:
        end_window = last_ts + dt.timedelta(hours=12)

# if df.empty:
#     st.info("Für dieses Zeitfenster liegen noch keine Day-Ahead-Daten vor.")
#     st.stop()

# # ---------------------------------------------------------
# # Komponenten: Spot + Gebühren (inkl. MwSt)
# # ---------------------------------------------------------
# fees = st.session_state.fees
# df["spot_ct"] = df["ct_per_kwh"]

# fees_no_vat = (
#     fees["stromsteuer_ct"]
#     + fees["umlagen_ct"]
#     + fees["konzessionsabgabe_ct"]
#     + fees["netzentgelt_ct"]
# )
# df["fees_incl_vat_ct"] = (fees_no_vat + df["spot_ct"]) * (fees["mwst"] / 100.0) + fees_no_vat
# df["total_ct"] = df["spot_ct"] + df["fees_incl_vat_ct"]


In [27]:
df_raw

,ts_ms,eur_per_mwh,ts,ct_per_kwh
0,1760306400000,93.45,2025-10-13 00:00:00+02:00,9.345
1,1760310000000,90.26,2025-10-13 01:00:00+02:00,9.026
2,1760313600000,90.83,2025-10-13 02:00:00+02:00,9.083
3,1760317200000,89.12,2025-10-13 03:00:00+02:00,8.912
4,1760320800000,93.45,2025-10-13 04:00:00+02:00,9.345
...,...,...,...,...
163,1760893200000,NaN,2025-10-19 19:00:00+02:00,NaN
164,1760896800000,NaN,2025-10-19 20:00:00+02:00,NaN
165,1760900400000,NaN,2025-10-19 21:00:00+02:00,NaN
166,1760904000000,NaN,2025-10-19 22:00:00+02:00,NaN


In [28]:
df

,ts_ms,eur_per_mwh,ts,ct_per_kwh
72,1760565600000,88.16,2025-10-16 00:00:00+02:00,8.816
73,1760569200000,85.81,2025-10-16 01:00:00+02:00,8.581
74,1760572800000,88.00,2025-10-16 02:00:00+02:00,8.800
75,1760576400000,86.66,2025-10-16 03:00:00+02:00,8.666
76,1760580000000,88.34,2025-10-16 04:00:00+02:00,8.834
77,1760583600000,85.05,2025-10-16 05:00:00+02:00,8.505
78,1760587200000,96.19,2025-10-16 06:00:00+02:00,9.619
79,1760590800000,116.12,2025-10-16 07:00:00+02:00,11.612
80,1760594400000,132.77,2025-10-16 08:00:00+02:00,13.277
81,1760598000000,119.93,2025-10-16 09:00:00+02:00,11.993


## Verschiedene Quellen

In [ ]:
# %reload_ext autoreload
# %autoreload 2

In [10]:
import price_sources

In [68]:
from datetime import datetime, timedelta, timezone
import pandas as pd

import importlib
importlib.reload(price_sources)
from price_sources import fetch_smard_day_ahead, _fmt_utc, DE_LU_EIC

## SMARD

In [75]:
# 1) SMARD (no key)
df_smard_15m = fetch_smard_day_ahead(resolution="PT15M")

In [79]:
df_smard_60m = fetch_smard_day_ahead(resolution="PT60M")

In [77]:
df_smard_15m

,ts,eur_per_mwh,ct_per_kwh,source,resolution
0,2025-10-20 00:00:00+02:00,82.95,8.295,SMARD,PT15M
1,2025-10-20 00:15:00+02:00,69.03,6.903,SMARD,PT15M
2,2025-10-20 00:30:00+02:00,70.04,7.004,SMARD,PT15M
3,2025-10-20 00:45:00+02:00,58.13,5.813,SMARD,PT15M
4,2025-10-20 01:00:00+02:00,70.30,7.030,SMARD,PT15M
...,...,...,...,...,...
187,2025-10-22 22:45:00+02:00,106.29,10.629,SMARD,PT15M
188,2025-10-22 23:00:00+02:00,114.50,11.450,SMARD,PT15M
189,2025-10-22 23:15:00+02:00,111.36,11.136,SMARD,PT15M
190,2025-10-22 23:30:00+02:00,107.41,10.741,SMARD,PT15M


In [80]:
df_smard_60m

,ts,eur_per_mwh,ct_per_kwh,source,resolution
0,2025-10-20 00:00:00+02:00,70.04,7.004,SMARD,PT60M
1,2025-10-20 01:00:00+02:00,63.98,6.398,SMARD,PT60M
2,2025-10-20 02:00:00+02:00,60.98,6.098,SMARD,PT60M
3,2025-10-20 03:00:00+02:00,60.00,6.000,SMARD,PT60M
4,2025-10-20 04:00:00+02:00,60.60,6.060,SMARD,PT60M
5,2025-10-20 05:00:00+02:00,74.81,7.481,SMARD,PT60M
6,2025-10-20 06:00:00+02:00,76.25,7.625,SMARD,PT60M
7,2025-10-20 07:00:00+02:00,91.87,9.187,SMARD,PT60M
8,2025-10-20 08:00:00+02:00,125.10,12.510,SMARD,PT60M
9,2025-10-20 09:00:00+02:00,92.19,9.219,SMARD,PT60M


## ENTSOE

In [92]:
import os
import tomllib as tomli # Use 'import tomllib' if you are on Python 3.11+

# --- Load ENTSO-E token from secrets.toml ---

entsoe_token = None
secrets_path = os.path.join(".streamlit", "secrets.toml")

try:
    with open(secrets_path, "rb") as f:
        secrets = tomli.load(f) # Use 'tomllib.load(f)' for Python 3.11+
        entsoe_token = secrets.get("entsoe_token")

    if entsoe_token:
        print("✅ ENTSO-E token loaded successfully.")
        # You can now use the 'entsoe_token' variable in your API calls
        # For example:
        # df = price_sources.fetch_entsoe_day_ahead(token=entsoe_token, ...)
    else:
        print("⚠️ 'entsoe_token' not found in the secrets file.")

except FileNotFoundError:
    print(f"❌ Error: Secrets file not found at '{secrets_path}'.")
    print("   Please ensure the path is correct relative to your notebook's location.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

✅ ENTSO-E token loaded successfully.


#### Check XML structure

In [26]:
import datetime as dt
import requests
from xml.etree import ElementTree as ET
ENTSOE_BASE = "https://web-api.tp.entsoe.eu/api"
DE_LU_EIC = "10Y1001A1001A82H"  # Bidding zone code for DE/LU (BZN)


def fetch_entsoe_day_ahead_test(
    token: str,
    start_utc: dt.datetime,
    end_utc: dt.datetime,
    eic_bzn: str = DE_LU_EIC,
) -> pd.DataFrame:
    """
    Fetch Day-Ahead prices via ENTSO-E REST (XML), returning:
      ts (Europe/Berlin), eur_per_mwh, ct_per_kwh, source="ENTSOE", resolution ("PT15M"/"PT60M").
    start_utc / end_utc must be tz-aware in any tz (converted to UTC for the query).
    """
    if start_utc.tzinfo is None or end_utc.tzinfo is None:
        raise ValueError("start_utc and end_utc must be timezone-aware")

    params = {
        "securityToken": token,
        "documentType": "A44",
        "in_Domain": eic_bzn,
        "out_Domain": eic_bzn,
        "periodStart": _fmt_utc(start_utc),
        "periodEnd": _fmt_utc(end_utc),
    }

    resp = requests.get(ENTSOE_BASE, params=params, timeout=45)
    if resp.status_code == 401:
        raise PermissionError("ENTSO-E: Unauthorized (check token)")
    resp.raise_for_status()

    return resp

In [27]:
# 2) ENTSO-E (needs token) – query yesterday→tomorrow to be safe; UTC times
now_utc = dt.datetime.now(timezone.utc)
yest_utc = now_utc - timedelta(days=1)
tom_utc  = now_utc + timedelta(days=2)
resp = fetch_entsoe_day_ahead_test(entsoe_token, yest_utc, tom_utc, eic_bzn=DE_LU_EIC)


[autoreload of price_sources failed: Traceback (most recent call last):
  File "c:\Users\ChristophPromberger\dev\strompreise-app\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 322, in check
    elif self.deduper_reloader.maybe_reload_module(m):
         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
  File "c:\Users\ChristophPromberger\dev\strompreise-app\.venv\Lib\site-packages\IPython\extensions\deduperreload\deduperreload.py", line 545, in maybe_reload_module
    new_source_code = f.read()
  File "C:\Python313\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 13165: character maps to <undefined>
]


ConnectTimeout: HTTPSConnectionPool(host='web-api.tp.entsoe.eu', port=443): Max retries exceeded with url: /api?securityToken=97721bb0-d6cd-4aa6-9bbc-36c9d95b205c&documentType=A44&in_Domain=10Y1001A1001A82H&out_Domain=10Y1001A1001A82H&periodStart=202510200917&periodEnd=202510230917 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000025C2FDF2C10>, 'Connection to web-api.tp.entsoe.eu timed out. (connect timeout=45)'))

In [24]:
# Parse XML
ns = {"ns": "urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3"}
try:
    root = ET.fromstring(resp.content)
except ET.ParseError as ex:
    raise RuntimeError(f"ENTSO-E: XML parse error: {ex}")

NameError: name 'resp' is not defined

In [8]:
rows = []
# Day-ahead doc can include multiple TimeSeries
for ts_node in root.findall(".//ns:TimeSeries", ns):
    rows.append(ts_node)

In [9]:
root

<Element '{urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3}Publication_MarketDocument' at 0x000001A5FF46CE50>

In [11]:
rows

[<Element '{urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3}TimeSeries' at 0x000001A5FF46EC00>,
 <Element '{urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3}TimeSeries' at 0x000001A5FF567970>,
 <Element '{urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3}TimeSeries' at 0x000001A5FF55D8F0>,
 <Element '{urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3}TimeSeries' at 0x000001A5FF557830>,
 <Element '{urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3}TimeSeries' at 0x000001A5FF53D7B0>]

In [32]:
import xml.dom.minidom as md

xml_pretty = md.parseString(resp.content).toprettyxml(indent="  ")
print(xml_pretty)  # show first 2000 chars

<?xml version="1.0" ?>
<Publication_MarketDocument xmlns="urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3">
  
    
  <mRID>9af8768d90704dd9842c65bebecff52f</mRID>
  
    
  <revisionNumber>1</revisionNumber>
  
    
  <type>A44</type>
  
    
  <sender_MarketParticipant.mRID codingScheme="A01">10X1001A1001A450</sender_MarketParticipant.mRID>
  
    
  <sender_MarketParticipant.marketRole.type>A32</sender_MarketParticipant.marketRole.type>
  
    
  <receiver_MarketParticipant.mRID codingScheme="A01">10X1001A1001A450</receiver_MarketParticipant.mRID>
  
    
  <receiver_MarketParticipant.marketRole.type>A33</receiver_MarketParticipant.marketRole.type>
  
    
  <createdDateTime>2025-10-20T14:32:13Z</createdDateTime>
  
    
  <period.timeInterval>
    
      
    <start>2025-10-18T22:00Z</start>
    
      
    <end>2025-10-20T22:00Z</end>
    
    
  </period.timeInterval>
  
      
  <TimeSeries>
    
        
    <mRID>1</mRID>
    
        
    <auction.type>A01</auction.ty

### Check ENTSOE data structure

In [67]:
import importlib
importlib.reload(price_sources)
from price_sources import fetch_entsoe_day_ahead   #re-import

In [57]:
now_utc = dt.datetime.now(timezone.utc)
from_utc = now_utc - timedelta(days=5)
to_utc  = now_utc + timedelta(days=5)
DE_LU_EIC = "10Y1001A1001A82H"  # Bidding zone code for DE/LU (BZN)

df_entsoe = fetch_entsoe_day_ahead(entsoe_token, from_utc, to_utc, eic_bzn=DE_LU_EIC)
df_entsoe['mrid'] = df_entsoe['mrid'].astype(int)

In [58]:
df_entsoe

,ts,eur_per_mwh,ct_per_kwh,source,resolution,mrid,position
0,2025-10-17 00:00:00+02:00,104.80,10.480,ENTSOE,PT15M,1,2
1,2025-10-17 00:00:00+02:00,91.40,9.140,ENTSOE,PT15M,2,1
2,2025-10-17 00:15:00+02:00,88.11,8.811,ENTSOE,PT15M,2,1
3,2025-10-17 00:15:00+02:00,96.70,9.670,ENTSOE,PT15M,1,2
4,2025-10-17 00:30:00+02:00,90.93,9.093,ENTSOE,PT15M,1,2
...,...,...,...,...,...,...,...
1037,2025-10-22 23:15:00+02:00,111.36,11.136,ENTSOE,PT15M,11,1
1038,2025-10-22 23:30:00+02:00,98.10,9.810,ENTSOE,PT15M,10,2
1039,2025-10-22 23:30:00+02:00,107.41,10.741,ENTSOE,PT15M,11,1
1040,2025-10-22 23:45:00+02:00,94.90,9.490,ENTSOE,PT15M,10,2


In [59]:
if not df_entsoe.empty:
    # Group by 'mrid' and aggregate to find the min and max timestamp for each group
    mrid_summary = df_entsoe.groupby(['position','mrid'])['ts'].agg(
        first_timestamp='min',
        last_timestamp='max'
    )
    
    print("First and last timestamp for each mRID in df_entsoe:")
    print(mrid_summary)
else:
    print("The DataFrame 'df_entsoe' is empty or has not been loaded yet.")

First and last timestamp for each mRID in df_entsoe:
                        first_timestamp            last_timestamp
position mrid                                                    
1        2    2025-10-17 00:00:00+02:00 2025-10-17 23:45:00+02:00
         4    2025-10-18 00:00:00+02:00 2025-10-18 23:45:00+02:00
         6    2025-10-19 00:00:00+02:00 2025-10-19 23:45:00+02:00
         8    2025-10-20 00:00:00+02:00 2025-10-20 23:45:00+02:00
         11   2025-10-22 00:00:00+02:00 2025-10-22 23:45:00+02:00
2        1    2025-10-17 00:00:00+02:00 2025-10-17 23:45:00+02:00
         3    2025-10-18 00:00:00+02:00 2025-10-18 23:45:00+02:00
         5    2025-10-19 00:00:00+02:00 2025-10-19 23:45:00+02:00
         7    2025-10-20 00:00:00+02:00 2025-10-20 23:45:00+02:00
         9    2025-10-21 00:00:00+02:00 2025-10-21 23:45:00+02:00
         10   2025-10-22 00:00:00+02:00 2025-10-22 23:45:00+02:00


In [73]:
df_smard_15m

,ts,eur_per_mwh,ct_per_kwh,source,resolution
0,2025-10-20 00:00:00+02:00,82.95,8.295,SMARD,PT15M
1,2025-10-20 00:15:00+02:00,69.03,6.903,SMARD,PT15M
2,2025-10-20 00:30:00+02:00,70.04,7.004,SMARD,PT15M
3,2025-10-20 00:45:00+02:00,58.13,5.813,SMARD,PT15M
4,2025-10-20 01:00:00+02:00,70.30,7.030,SMARD,PT15M
...,...,...,...,...,...
187,2025-10-22 22:45:00+02:00,106.29,10.629,SMARD,PT15M
188,2025-10-22 23:00:00+02:00,114.50,11.450,SMARD,PT15M
189,2025-10-22 23:15:00+02:00,111.36,11.136,SMARD,PT15M
190,2025-10-22 23:30:00+02:00,107.41,10.741,SMARD,PT15M


In [81]:
df_smard_60m


,ts,eur_per_mwh,ct_per_kwh,source,resolution
0,2025-10-20 00:00:00+02:00,70.04,7.004,SMARD,PT60M
1,2025-10-20 01:00:00+02:00,63.98,6.398,SMARD,PT60M
2,2025-10-20 02:00:00+02:00,60.98,6.098,SMARD,PT60M
3,2025-10-20 03:00:00+02:00,60.00,6.000,SMARD,PT60M
4,2025-10-20 04:00:00+02:00,60.60,6.060,SMARD,PT60M
5,2025-10-20 05:00:00+02:00,74.81,7.481,SMARD,PT60M
6,2025-10-20 06:00:00+02:00,76.25,7.625,SMARD,PT60M
7,2025-10-20 07:00:00+02:00,91.87,9.187,SMARD,PT60M
8,2025-10-20 08:00:00+02:00,125.10,12.510,SMARD,PT60M
9,2025-10-20 09:00:00+02:00,92.19,9.219,SMARD,PT60M


In [63]:
df_entsoe

,ts,eur_per_mwh,ct_per_kwh,source,resolution,mrid,position
0,2025-10-17 00:00:00+02:00,104.80,10.480,ENTSOE,PT15M,1,2
1,2025-10-17 00:00:00+02:00,91.40,9.140,ENTSOE,PT15M,2,1
2,2025-10-17 00:15:00+02:00,88.11,8.811,ENTSOE,PT15M,2,1
3,2025-10-17 00:15:00+02:00,96.70,9.670,ENTSOE,PT15M,1,2
4,2025-10-17 00:30:00+02:00,90.93,9.093,ENTSOE,PT15M,1,2
...,...,...,...,...,...,...,...
1037,2025-10-22 23:15:00+02:00,111.36,11.136,ENTSOE,PT15M,11,1
1038,2025-10-22 23:30:00+02:00,98.10,9.810,ENTSOE,PT15M,10,2
1039,2025-10-22 23:30:00+02:00,107.41,10.741,ENTSOE,PT15M,11,1
1040,2025-10-22 23:45:00+02:00,94.90,9.490,ENTSOE,PT15M,10,2


In [90]:
# Plot the different time series ---
if not df_entsoe.empty:
    fig = go.Figure()

    # Define a color map to assign a unique color to each mRID
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']
    unique_mrids = sorted(df_entsoe['mrid'].unique())
    color_map = {mrid: colors[i % len(colors)] for i, mrid in enumerate(unique_mrids)}

    # Group by the 'mrid' column and create a trace for each group
    for (position, mrid), group_df in df_entsoe.groupby(['position','mrid']):
        
        # To fix the "overshoot" issue with step charts, add a final point
        # to explicitly define the end of the last time interval.
        plot_df = group_df.copy()
        if not plot_df.empty:
            last_row = plot_df.iloc[-1:].copy()
            if len(plot_df) > 1:
                time_diff = plot_df['ts'].iloc[-1] - plot_df['ts'].iloc[-2]
            else:
                # Guess interval if only one point exists
                time_diff = pd.Timedelta(minutes=15)
            last_row['ts'] = last_row['ts'] + time_diff
            plot_df = pd.concat([plot_df, last_row], ignore_index=True)
        
        fig.add_trace(go.Scatter(
            x=plot_df['ts'],
            y=plot_df['ct_per_kwh'],
            mode='lines',
            line_shape='hv',  # Vertical-then-horizontal step chart
            line=dict(color=color_map.get(mrid)), # Assign a specific color
            name=f"pos/mRID: {position}/{mrid}",  # Name the trace using the mRID
            hovertemplate=f"pos/mRID {position}/{mrid}: " + "%{y:.2f} ct/kWh<extra></extra>"
        ))
# Chart for SMARD 15m
    fig.add_trace(go.Scatter(
            x=df_smard_15m['ts'],
            y=df_smard_15m['ct_per_kwh'],
            mode='lines',
            line_shape='hv',  # Vertical-then-horizontal step chart
            line=dict(width=2, color="black", dash="dash"), # Assign a specific color
            name=f"SMARD 15m",  # Name the trace using the mRID
            hovertemplate=f"SMARD 15m: " + "%{y:.2f} ct/kWh<extra></extra>"
        ))
# Chart for SMARD 60m
    fig.add_trace(go.Scatter(
                x=df_smard_60m['ts'],
                y=df_smard_60m['ct_per_kwh'],
                mode='lines',
                line_shape='hv',  # Vertical-then-horizontal step chart
                line=dict(width=2, color="blue", dash="dashdot"), # Assign a specific color
                name=f"SMARD 60m",  # Name the trace using the mRID
                hovertemplate=f"SMARD 60m: " + "%{y:.2f} ct/kWh<extra></extra>"
            ))

    fig.update_layout(
        title="Day-Ahead Prices by Source / Time Series (mRID)",
        xaxis_title="Time (Europe/Berlin)",
        yaxis_title="Price (ct/kWh)",
        hovermode="x unified",
        legend_title="Time Series"
    )

    fig.show()
else:
    print("DataFrame is empty, cannot generate plot.")

#### Combine positions 1 and 2

In [94]:
now_utc = dt.datetime.now(timezone.utc)
from_utc = now_utc - timedelta(days=5)
to_utc  = now_utc + timedelta(days=5)
DE_LU_EIC = "10Y1001A1001A82H"  # Bidding zone code for DE/LU (BZN)

df_raw, data_meta = load_price_data(
    source="ENTSOE-COMBINED",
    resolution="PT15M",
    entsoe_token=entsoe_token,
    entsoe_days_back=3,
    entsoe_days_forward=1
    )

In [ ]:
importlib.reload(app)
from app import prepare_price_dataframe, estimate_fees_from_plz

In [125]:
fees = estimate_fees_from_plz("82340")
df_all = prepare_price_dataframe(df_raw, fees)

In [126]:
df_all

,ts,eur_per_mwh,ct_per_kwh,source,position,mrid,resolution,spot_ct,fees_excl_vat_ct,vat_ct,fees_incl_vat_ct,total_ct
0,2025-10-19 00:00:00+02:00,103.22,10.322,ENTSOE,1,2,PT15M,10.322,14.45,4.70668,19.15668,29.47868
1,2025-10-19 00:15:00+02:00,92.00,9.200,ENTSOE,1,2,PT15M,9.200,14.45,4.49350,18.94350,28.14350
2,2025-10-19 00:30:00+02:00,87.58,8.758,ENTSOE,1,2,PT15M,8.758,14.45,4.40952,18.85952,27.61752
3,2025-10-19 00:45:00+02:00,86.57,8.657,ENTSOE,1,2,PT15M,8.657,14.45,4.39033,18.84033,27.49733
4,2025-10-19 01:00:00+02:00,95.65,9.565,ENTSOE,1,2,PT15M,9.565,14.45,4.56285,19.01285,28.57785
...,...,...,...,...,...,...,...,...,...,...,...,...
375,2025-10-22 22:45:00+02:00,106.29,10.629,ENTSOE,1,7,PT15M,10.629,14.45,4.76501,19.21501,29.84401
376,2025-10-22 23:00:00+02:00,114.50,11.450,ENTSOE,1,7,PT15M,11.450,14.45,4.92100,19.37100,30.82100
377,2025-10-22 23:15:00+02:00,111.36,11.136,ENTSOE,1,7,PT15M,11.136,14.45,4.86134,19.31134,30.44734
378,2025-10-22 23:30:00+02:00,107.41,10.741,ENTSOE,1,7,PT15M,10.741,14.45,4.78629,19.23629,29.97729


In [164]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from typing import Optional, Mapping, Dict, Any, Iterable, Callable, Union

def plot_segments_by_category(
    df: pd.DataFrame,
    *,
    time_col: str = "timestamp",
    value_col: str = "value",
    category_col: str = "category",
    params_by_category: Mapping[str, Dict[str, Any]],
    scatter_defaults: Optional[Dict[str, Any]] = None,  # global defaults for every trace
    sort_by_time: bool = True,
    layout: Optional[Dict[str, Any]] = None,
    fig: Optional[go.Figure] = None
) -> go.Figure:
    """
    Render a time series as contiguous segments per `category`, each segment as its own go.Scatter trace.

    Features
    --------
    - Per-category params deep-merge over `scatter_defaults` (recursive for dicts like line/marker).
    - Robust `customdata` handling per segment:
        * "col:<name>" or "<name>" -> single column
        * [col1, col2, ...]        -> multiple columns
        * pd.Series / np.ndarray   -> vector(s) aligned to the (sorted/filtered) df
        * callable(seg_df)         -> custom builder returning (n,) or (n,k)
    - `fill="tonexty"` support via `tonexty_anchor` per category:
        * "zero"  -> baseline at 0
        * "prev"  -> baseline at last y of previous segment with same category
        * "<col>" -> baseline from a column in df
        * callable(seg_df) -> baseline array
      A hidden baseline trace is inserted immediately before the filled trace.

    Parameters
    ----------
    df : DataFrame with columns [time_col, value_col, category_col]
    time_col, value_col, category_col : str
    params_by_category : dict[str, dict]
        Per-category overrides for go.Scatter kwargs. May include:
          - customdata : see above
          - tonexty_anchor : "zero" | "prev" | "<colname>" | callable(seg_df)->array
    scatter_defaults : dict | None
        Global defaults for go.Scatter (e.g., mode, line, hovertemplate, line_shape, etc.).
    sort_by_time : bool
    layout : dict | None
        Passed to fig.update_layout(**layout).
    fig : go.Figure | None
        Existing figure to append to.

    Returns
    -------
    go.Figure
    """

    # ---------------- helper functions (scoped) ----------------
    CustomDataSpec = Union[
        None, str, Iterable[str], pd.Series, np.ndarray, Callable[[pd.DataFrame], np.ndarray]
    ]

    def _deep_merge(base: Dict[str, Any], override: Dict[str, Any]) -> Dict[str, Any]:
        """Recursively merge dicts; override wins. Non-dicts (lists/arrays/strings) are replaced."""
        out = dict(base or {})
        for k, v in (override or {}).items():
            if k in out and isinstance(out[k], dict) and isinstance(v, dict):
                out[k] = _deep_merge(out[k], v)
            else:
                out[k] = v
        return out

    def _resolve_segment_customdata(seg: pd.DataFrame, full_df: pd.DataFrame, spec: CustomDataSpec):
        """Return per-segment customdata ndarray or None."""
        if spec is None:
            return None

        if callable(spec):
            arr = spec(seg)
            return np.asarray(arr) if arr is not None else None

        if isinstance(spec, str):
            col = spec[4:] if spec.startswith("col:") else spec
            if col not in seg.columns:
                raise KeyError(f"customdata column '{col}' not in DataFrame segment.")
            return seg[[col]].to_numpy()  # (n,1)

        if isinstance(spec, (list, tuple)) and all(isinstance(c, str) for c in spec):
            missing = [c for c in spec if c not in seg.columns]
            if missing:
                raise KeyError(f"customdata columns missing: {missing}")
            return seg[list(spec)].to_numpy()  # (n,k)

        # Vector-like aligned to FULL (sorted/filtered) df
        if isinstance(spec, pd.Series):
            vec = spec.to_numpy()
        elif isinstance(spec, np.ndarray):
            vec = spec
        else:
            raise TypeError(
                "Unsupported customdata spec. Use a column name, list of columns, "
                "Series/ndarray aligned to the sorted df, or a callable(seg_df)->array."
            )

        if vec.shape[0] != len(full_df):
            raise ValueError(f"customdata vector must have length {len(full_df)} (got {vec.shape[0]}).")

        pos_idx = seg["_rowid"].to_numpy()  # 0..n-1 positional ids
        arr = vec[pos_idx]
        return arr.reshape(-1, 1) if arr.ndim == 1 else np.asarray(arr)

    TonextyAnchor = Union[str, Callable[[pd.DataFrame], Iterable[float]]]

    def _resolve_tonexty_baseline(
        seg: pd.DataFrame,
        *,
        anchor: TonextyAnchor,
        last_cat_value: Optional[float]
    ) -> Optional[np.ndarray]:
        """Return (n,) baseline array for tonexty or None."""
        n = len(seg)
        if anchor is None:
            return None
        if isinstance(anchor, str):
            if anchor == "zero":
                return np.zeros(n, dtype=float)
            if anchor == "prev":
                base = 0.0 if last_cat_value is None else float(last_cat_value)
                return np.full(n, base, dtype=float)
            # treat as column name
            if anchor not in seg.columns:
                raise KeyError(f"tonexty_anchor column '{anchor}' not in segment.")
            return seg[anchor].to_numpy(dtype=float)
        # callable: build from segment df
        arr = np.asarray(anchor(seg), dtype=float)
        if arr.shape[0] != n:
            raise ValueError("tonexty_anchor callable must return array of same length as segment.")
        return arr

    # ---------------- validation & prep ----------------
    for c in (time_col, value_col, category_col):
        if c not in df.columns:
            raise ValueError(f"DataFrame missing column: {c}")

    _df = df.dropna(subset=[time_col, value_col]).copy()
    if sort_by_time:
        _df = _df.sort_values(time_col)
    _df[category_col] = _df[category_col].astype(str)

    # Stable positional id for alignment (0..n-1)
    _df = _df.reset_index(drop=True)
    _df["_rowid"] = np.arange(len(_df))

    # Contiguous segments
    _df["_segment_id"] = (_df[category_col] != _df[category_col].shift(1)).cumsum()

    fig = fig or go.Figure()
    legend_seen = {getattr(tr, "name", None) for tr in fig.data if hasattr(tr, "name")}

    # Track last value per category (for anchor='prev')
    last_value_by_cat: Dict[str, float] = {}

    # ---------------- build traces ----------------
    for _, seg in _df.groupby("_segment_id", sort=True):
        cat = seg[category_col].iloc[0]
        per_cat = params_by_category.get(cat, {})
        if not (per_cat or scatter_defaults):
            continue

        merged = _deep_merge(scatter_defaults or {}, per_cat or {})
        customdata_spec = merged.pop("customdata", None)
        tonexty_anchor = merged.pop("tonexty_anchor", None)

        merged.setdefault("name", cat)
        if "showlegend" not in merged:
            merged["showlegend"] = (merged["name"] not in legend_seen)
        legend_seen.add(merged["name"])

        # If tonexty requested, insert baseline first
        wants_tonexty = (merged.get("fill") == "tonexty") and (tonexty_anchor is not None)
        if wants_tonexty:
            baseline_y = _resolve_tonexty_baseline(
                seg,
                anchor=tonexty_anchor,
                last_cat_value=last_value_by_cat.get(cat)
            )
            if baseline_y is not None:
                fig.add_trace(go.Scatter(
                    x=seg[time_col],
                    y=baseline_y,
                    mode="lines",
                    line=dict(width=0),
                    hoverinfo="skip",
                    showlegend=False,
                    name=f"__baseline__{merged['name']}",
                    legendgroup=merged.get("legendgroup", None)
                ))

        seg_customdata = _resolve_segment_customdata(seg, _df, customdata_spec)

        fig.add_trace(go.Scatter(
            x=seg[time_col],
            y=seg[value_col],
            customdata=seg_customdata,
            **merged
        ))

        last_value_by_cat[cat] = float(seg[value_col].iloc[-1])

    # Layout
    fig.update_layout(xaxis_title=time_col, yaxis_title=value_col)
    if layout:
        fig.update_layout(**layout)
    return fig


In [172]:
# Base hovertemplate for spot price
# spot_hovertemplate = "Börsenstrompreis: %{y:.1f} ct/kWh<extra></extra>"
# custom_data_spot = None

spot_hovertemplate = (
        "Börsenstrompreis: %{y:.1f} ct/kWh<br>" +
        "ENTSO-E Position: %{customdata}<extra></extra>"
    )

default_params = dict(
        name="Börsenstrompreis", 
        mode="lines",
        line_shape="hv", 
        line=dict(width=0.8, color="#1f77b4"),
        fill="tozeroy",
        fillcolor="rgba(31, 119, 180, 0.5)",
        customdata="position",
        hovertemplate=spot_hovertemplate
)

params_by_position = {
    "1": dict(
        line=dict(width=0.8, color="#1f77b4")
    ),
    "2": dict(
        line=dict(width=0.8, color="#1f77b4", dash="dash"),
        fillcolor="rgba(31, 119, 180, 0.25)",
    )
}
fig = plot_segments_by_category(df_all, time_col="ts", value_col="ct_per_kwh", category_col="position", 
                                 scatter_defaults=default_params, params_by_category=params_by_position)


In [173]:
# fig

In [174]:
# Total prices
default_params = dict(
    name="Gesamtpreis", mode="lines",
    line_shape="hv", line=dict(width=1.2, color="#d62728"),
    fill="tonexty", tonexty_anchor="ct_per_kwh", 
    fillcolor="rgba(255, 127, 14, 0.5)",
    customdata="fees_incl_vat_ct",
    hovertemplate="Gesamtpreis: %{y:.1f} ct/kWh<br>Gebühren: %{customdata:.1f} ct/kWh<extra></extra>"
)

params_by_position = {
    "1": dict(),
    "2": dict(
        line=dict(dash="dash"),
        fillcolor="rgba(255, 127, 14, 0.25)",
    )
}

fig = plot_segments_by_category(df_all, time_col="ts", value_col="total_ct", category_col="position", 
                                fig=fig,
                                scatter_defaults=default_params, params_by_category=params_by_position)


fig.update_layout(
    height=400,
    margin=dict(l=10, r=10, t=10, b=10),
    # xaxis=dict(
    #     title="Zeit (lokal)",
    #     type="date",
    #     range=[range_start, range_end],
    #     rangeslider=dict(visible=True),
    #     fixedrange=True,
    # ),
    # yaxis=dict(
    #     title="ct/kWh",
    #     range=[0.0, y_max + padding],
    # ),
    hovermode="x unified",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1.0,
    ),
    hoverlabel=dict(bgcolor="rgba(255,255,255,0.9)", namelength=-1),
    # shapes=day_shapes,
)

In [44]:
df

,ts,eur_per_mwh,ct_per_kwh,source,resolution,mrid,position
0,2025-10-16 00:00:00+02:00,90.22,9.022,ENTSOE,PT15M,1,1
3,2025-10-16 00:15:00+02:00,91.63,9.163,ENTSOE,PT15M,1,1
5,2025-10-16 00:30:00+02:00,88.93,8.893,ENTSOE,PT15M,1,1
6,2025-10-16 00:45:00+02:00,81.88,8.188,ENTSOE,PT15M,1,1
8,2025-10-16 01:00:00+02:00,90.67,9.067,ENTSOE,PT15M,1,1
...,...,...,...,...,...,...,...
1132,2025-10-22 22:45:00+02:00,102.86,10.286,ENTSOE,PT15M,12,2
1133,2025-10-22 23:00:00+02:00,120.30,12.030,ENTSOE,PT15M,12,2
1134,2025-10-22 23:15:00+02:00,107.40,10.740,ENTSOE,PT15M,12,2
1135,2025-10-22 23:30:00+02:00,98.10,9.810,ENTSOE,PT15M,12,2


## smart Energy

In [ ]:

# 3) smartENERGY (if/when you have their JSON endpoint)
# Example placeholders – update url/fields once you have the real endpoint
# url = "https://www.smartenergy.at/api/spot"
# df_se = fetch_smartenergy(url, ts_field="timestamp", price_field="marketprice_eur_mwh", tz="Europe/Vienna")

# Compare coverage recency
def summarize(df, name):
    if df.empty:
        return f"{name}: empty"
    return f"{name}: {df['ts'].min()} → {df['ts'].max()} ({len(df)} rows)"

print(summarize(df_smard, "SMARD"))
print(summarize(df_entsoe, "ENTSO-E"))
# print(summarize(df_se, "smartENERGY"))

In [23]:
import price_sources

In [24]:
def load_price_data(
    source: str,
    resolution: str,
    entsoe_token: Optional[str],
    entsoe_days_back: int,
    entsoe_days_forward: int,
) -> tuple[pd.DataFrame, dict]:
    if source == "SMARD":
        df = price_sources.fetch_smard_day_ahead(resolution=resolution)
        meta = {"region": "DE-LU", "raw_resolution": resolution, "source_id": "SMARD"}
        return df, meta
    if source == "ENTSOE":
        if not entsoe_token:
            raise PriceDataError("Für ENTSO-E wird ein API Token benötigt.")
        now_utc = dt.datetime.now(dt.timezone.utc)
        start_utc = (now_utc - dt.timedelta(days=entsoe_days_back)).replace(minute=0, second=0, microsecond=0)
        end_utc = (now_utc + dt.timedelta(days=entsoe_days_forward)).replace(minute=0, second=0, microsecond=0)
        df = price_sources.fetch_entsoe_day_ahead(
            token=entsoe_token,
            start_utc=start_utc,
            end_utc=end_utc,
        )
        meta = {"region": "DE-LU", "raw_resolution": None, "source_id": "ENTSOE"}
        return df, meta
    raise PriceDataError(f"Unbekannte Datenquelle: {source}")

In [25]:
class PriceDataError(Exception):
    """Raised when a selected data source cannot provide usable data."""

data_source_choice = "ENTSOE"
resolution_choice = "quarterhour"
entsoe_token = "97721bb0-d6cd-4aa6-9bbc-36c9d95b205c"

try:
    bdf_raw, data_meta = load_price_data(
        source=data_source_choice,
        resolution=resolution_choice,
        entsoe_token=st.session_state.entsoe_token,
        entsoe_days_back=st.session_state.entsoe_days_back,
        entsoe_days_forward=st.session_state.entsoe_days_forward,
    )
except PriceDataError as exc:
    st.error(f"⚠️ {exc}")
    st.stop()
except Exception as exc:
    st.error(f"⚠️ Unerwarteter Fehler beim Laden der Datenquelle: {exc}")
    st.stop()
#used_region = data_meta.get("region", "DE-LU")
# resolution_fallback = normalize_resolution_hint(
    # data_meta.get("raw_resolution"), default=resolution_choice
# )

2025-10-20 16:21:35.870 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-20 16:21:35.871 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-20 16:21:35.872 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-20 16:21:35.873 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-20 16:21:35.874 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [26]:
df_raw, data_meta = load_price_data(
        source=data_source_choice,
        resolution=resolution_choice,
        entsoe_token=entsoe_token,
        entsoe_days_back=2,
        entsoe_days_forward=1,
    )

In [27]:
df_raw

,ts,eur_per_mwh,ct_per_kwh,source,resolution
0,2025-10-18 00:00:00+02:00,97.59,9.759,ENTSOE,PT15M
1,2025-10-18 00:00:00+02:00,93.61,9.361,ENTSOE,PT15M
2,2025-10-18 00:15:00+02:00,92.60,9.260,ENTSOE,PT15M
3,2025-10-18 00:15:00+02:00,103.62,10.362,ENTSOE,PT15M
4,2025-10-18 00:30:00+02:00,96.77,9.677,ENTSOE,PT15M
...,...,...,...,...,...
659,2025-10-21 22:45:00+02:00,67.76,6.776,ENTSOE,PT15M
660,2025-10-21 23:00:00+02:00,79.80,7.980,ENTSOE,PT15M
661,2025-10-21 23:15:00+02:00,71.45,7.145,ENTSOE,PT15M
662,2025-10-21 23:30:00+02:00,71.30,7.130,ENTSOE,PT15M
